# Introduction

This notebook demonstrates the creation of a synthetic population using an Iterative Proportional Fitting (IPF) approach.

IPF is a statistical technique that tries to adjust the values of a matrix (joint distribution) in order to match the values of expected single distributions (marginals) for each matrix dimension. We use this approach to sample from a set of persons with certain demographic attributes in a way that the observed distributions in each zone (ie from census) are met.

In [1]:
import itertools

import pandas as pd
from pam.core import Person, Population
from pam.planner import ipf

ModuleNotFoundError: No module named 'pam'

# Data

We start with some demographic data for each zone, as shown below. The zones dataframe includes the zone name as the index, and its columns follow a `variable|class` naming convention. Alternatively, they could be provided with a mutltiIndex column, with the first level being the variable and the second level indicating the class. The controlled variables should be part of the seed population's attributes.

In [2]:
zone_data = pd.DataFrame(
    {
        "zone": ["a", "b"],
        "hhincome|high": [30, 80],
        "hhincome|medium": [40, 100],
        "hhincome|low": [30, 20],
        "age|minor": [40, 90],
        "age|adult": [60, 110],
        "carAvail|yes": [90, 180],
        "carAvail|no": [10, 20],
    }
).set_index("zone")
zone_data

Let's create a seed population which includes every possible combination of attributes:

In [3]:
dims = {"hhincome": ["low", "medium", "high"], "age": ["minor", "adult"], "carAvail": ["yes", "no"]}
list(itertools.product(*dims.values()))

# %%
seed_pop = Population()
n = 0
for attributes in list(itertools.product(*dims.values())):
    hhincome, age, carAvail = attributes
    person = Person(
        pid=n, attributes={"hhincome": hhincome, "age": age, "carAvail": carAvail, "hzone": pd.NA}
    )
    n += 1
    seed_pop.add(person)

seed_pop.random_person().attributes

# IPF

Now let's create a population that matches the demographic distribution of each zone:

In [4]:
pop = ipf.generate_population(seed_pop, zone_data)

The resulting population comprises 300 persons (as defined in the zone data):

In [5]:
len(pop)

And each person in the population is assigned a household zone:

In [6]:
pop.random_person().attributes

The resulting joint demographic distributions in each zone are shown below:

In [7]:
summary = (
    pd.DataFrame([person.attributes for hid, pid, person in pop.people()])
    .value_counts()
    .reorder_levels([3, 0, 1, 2])
    .sort_index()
)

summary

The aggregate demographic distributions match the marginals in `zone_data`:

In [8]:
summary_aggregate = []
for var in dims:
    df = summary.groupby(level=["hzone", var]).sum().unstack(level=var)
    df.columns = [f"{var}|{x}" for x in df.columns]
    summary_aggregate.append(df)
summary_aggregate = pd.concat(summary_aggregate, axis=1)
summary_aggregate.index.name = "zone"
summary_aggregate = summary_aggregate[zone_data.columns]
summary_aggregate

In [9]:
pd.testing.assert_frame_equal(
    summary_aggregate, zone_data, check_exact=False, atol=1
)  # test passes